In [15]:
import re
import os

def parse_log_file(log_filepath):
    """
    解析日志文件，精确提取与每个评估周期相关的特定行。
    它识别由 'Loaded weights from checkpoint' 开始的块，并仅提取
    该周期内的评估指标和摘要行。

    Args:
        log_filepath (str): 日志文件的路径。

    Returns:
        list: 一个字符串列表，每个字符串是需要保留的一行。
    """
    # 正则表达式列表，用于匹配需要保留的行的内容
    keep_patterns = [
        re.compile(r'Loaded weights from checkpoint:.*ckpt\.iter\d+\.pth'),
        re.compile(r'Agent parameters:'),
        re.compile(r'Finished setting up policy'),
        re.compile(r'Episodes evaluated:'),
        re.compile(r'Average episode success:'), # 匹配所有 "Average episode..." 的行
        re.compile(r'Average episode spl:'), # 匹配所有 "Average episode..." 的行
        re.compile(r'^Finish ')          # 匹配以 'Finish ' 开头的行
    ]

    # 这个正则表达式只用来识别一个新评估块的开始
    checkpoint_pattern = re.compile(r'Loaded weights from checkpoint:.*ckpt\.iter\d+\.pth')

    extracted_lines = []
    # 状态标志：是否处于一个我们感兴趣的评估块中
    in_evaluation_block = False

    try:
        with open(log_filepath, 'r', encoding='utf-8') as f:
            for line in f:
                stripped_line = line.strip()
                if not stripped_line: # 跳过空行
                    continue

                # 检查这一行是否是一个新评估块的开始
                is_start_of_block = checkpoint_pattern.search(stripped_line)
                if is_start_of_block:
                    in_evaluation_block = True
                    # 如果不是第一个块，则在块之间添加一个空行作为分隔
                    if extracted_lines and extracted_lines[-1] != '\n':
                         extracted_lines.append('\n')

                # 如果我们正处于一个评估块中...
                if in_evaluation_block:
                    # ...检查当前行是否是我们需要保留的行
                    should_keep = False
                    for pattern in keep_patterns:
                        if pattern.search(stripped_line):
                            should_keep = True
                            break
                    
                    if should_keep:
                        extracted_lines.append(line)

                    # 如果读到了 "Finish" 行，说明这个评估块结束了
                    if stripped_line.startswith('Finish'):
                        in_evaluation_block = False

    except FileNotFoundError:
        print(f"错误：文件 '{log_filepath}' 未找到。")
        return []
    except Exception as e:
        print(f"处理文件时发生错误: {e}")
        return []

    return extracted_lines

def save_to_txt(data, output_filepath):
    """
    将提取的行原样保存到 TXT 文件中。

    Args:
        data (list): 从日志中提取的行列表。
        output_filepath (str): 输出 TXT 文件的路径。
    """
    if not data:
        print("没有可供保存的数据。")
        return
    
    try:
        with open(output_filepath, 'w', encoding='utf-8') as txtfile:
            # writelines 可以直接将列表中的所有行写入文件
            txtfile.writelines(data)
        print(f"数据已成功保存到: {output_filepath}")
    except Exception as e:
        print(f"保存 TXT 文件时发生错误: {e}")


# --- 使用示例 ---
# 1. 请确保您的日志文件与此脚本在同一目录下，或提供完整路径。
#    将下面的 log_file_path 值替换成您的实际文件名。
log_file_path = 'evalall_r2r_v13_train_navc(envdrop)_1_finetune.out'

# 2. 指定您想保存的 TXT 文件名
output_txt_path = 'evalall_r2r_v13_train_navc(envdrop)_1_finetune.txt'

# 3. 运行解析和保存函数
print(f"正在读取日志文件: {log_file_path}")
extracted_data = parse_log_file(log_file_path)

if extracted_data:
    save_to_txt(extracted_data, output_txt_path)
    # (可选) 打印输出文件的内容进行验证
    print(f"\n--- 已提取内容并保存到 '{output_txt_path}' ---")
    with open(output_txt_path, 'r', encoding='utf-8') as f:
        print(f.read())
else:
    print("在日志文件中未找到匹配的评估块。")



正在读取日志文件: evalall_r2r_v13_train_navc(envdrop)_1_finetune.out
数据已成功保存到: evalall_r2r_v13_train_navc(envdrop)_1_finetune.txt

--- 已提取内容并保存到 'evalall_r2r_v13_train_navc(envdrop)_1_finetune.txt' ---
2025-06-21 11:54:44,311 Loaded weights from checkpoint: data/logs/checkpoints/r2r_v13_train_navc(envdrop)_1_finetune/ckpt.iter12000.pth, iteration: 12000
2025-06-21 11:54:44,315 Agent parameters: 299.14 MB. Trainable: 140.79 MB.
2025-06-21 11:54:44,316 Finished setting up policy.
2025-06-21 12:04:04,182 Episodes evaluated: 1839
2025-06-21 12:04:04,231 Average episode success: 0.559543
2025-06-21 12:04:04,232 Average episode spl: 0.459480

2025-06-21 12:05:52,476 Loaded weights from checkpoint: data/logs/checkpoints/r2r_v13_train_navc(envdrop)_1_finetune/ckpt.iter15000.pth, iteration: 15000
2025-06-21 12:05:52,484 Agent parameters: 299.14 MB. Trainable: 140.79 MB.
2025-06-21 12:05:52,484 Finished setting up policy.
2025-06-21 12:14:52,959 Episodes evaluated: 1839
2025-06-21 12:14:52,998 Average e